In [1]:
class Matrice:
    """
    Ceci est une matrice.
    
    Passer une liste de coefficents au constructeur.
    """
    def __init__(self, coefficients):
        if not isinstance(coefficients, list):
            raise RuntimeError("Pas une liste")
        if not len(coefficients) > 0:
            raise RuntimeError("Matrice vide")
        if not all(isinstance(ligne, list)
                   for ligne in coefficients):
            raise RuntimeError("Pas une liste de listes")
            
        self.nblignes = len(coefficients)
        self.nbcolonnes = len(coefficients[0])
        
        if not all(len(ligne) == self.nbcolonnes
                   for ligne in coefficients):
            raise RuntimeError("Longueurs des lignes différentes")
        self.coeffs = coefficients
   
    def __repr__(self):
        # On mesure la largeur maximale des colonnes
        largeur = max(max(len(str(c)) for c in ligne) for ligne in self.coeffs)
        
        # On compose la sortie. On fait un large usage des fonctions de formatage
        # https://docs.python.org/3.5/library/stdtypes.html#printf-style-string-formatting
        resultat = ""
        for (i, ligne) in enumerate(self.coeffs):
            if i == 0:
                template = "/ %s \\\n" 
            elif i < len(self.coeffs) - 1:
                template = "| %s |\n" 
            else:
                template = "\\ %s /"
            resultat += template % " ".join(("%%%dd" % largeur) % c for c in ligne)
        return resultat
    
    def __eq__(self, other):
        "Méthode spéciale pour tester l'égalité de matrices"
        if not isinstance(other, Matrice):
            return False
        if self.nblignes != other.nblignes or self.nbcolonnes != other.nbcolonnes:
            return False
        return all(sc == oc 
                   for sligne, oligne in zip(self.coeffs, other.coeffs)
                   for sc, oc in zip(sligne, oligne))

    def __add__(self, other):
        "Méthode spéciale pour additionner deux matrices"
        if not isinstance(other, Matrice):
            raise TypeError("Pas une matrice")
        if self.nblignes != other.nblignes or self.nbcolonnes != other.nbcolonnes:
            raise RuntimeError("Pas la bonne taille")
        return Matrice([[sc + oc for sc, oc in zip(sligne, oligne)]
                         for sligne, oligne in zip(self.coeffs, other.coeffs)])

    def __neg__(self):
        "Méthode spéciale pour calculer l'opposée d'une matrice"
        return Matrice([[-c for c in ligne] for ligne in self.coeffs])
    
    def __sub__(self, other):
        "Méthode spéciale pour soustraire deux matrices"
        return self + (-other)
    
    def __mul__(self, other):
        "Méthode spéciale pour multiplier deux matrices"
        if not isinstance(other, Matrice):
            raise TypeError("Pas une matrice")
        if self.nbcolonnes != other.nblignes:
            raise RuntimeError("Pas la bonne taille")
            
        # Première solution, en initialisant un tableau de 0 à deux dimensions.
        # Pas très Pythonique...
        matrice = [[0 for i in range(other.nbcolonnes)]
                   for j in range(self.nblignes)]
        for i in range(self.nblignes):
            for j in range(other.nbcolonnes):
                matrice[i][j] = sum(self.coeffs[i][k]*other.coeffs[k][j] 
                                    for k in range(self.nbcolonnes))
        return Matrice(matrice)

        # Deuxième solution, avec un tableau qui grandit au fur et à mesure
#        matrice = []
#        for i in range(self.nblignes):
#            lignes = []
#            for j in range(other.nbcolonnes):
#                ligne.append(sum(...))
#            matrice.append(ligne)
#        return Matrice(matrice)


        # Troisième solution, avec des compréhensions de listes
#        return Matrice([
#                        [
#                         sum(self.coeffs[i][k]*other.coeffs[k][j] 
#                             for k in range(self.nbcolonnes))
#                         for j in range(other.nbcolonnes)
#                        ]
#                        for i in range(self.nblignes)
#                       ])


    def strassen(self, other, threshold=1):
        """Multiplication de Strassen.
        `threshold` indique à quelle taille arrêter la récurrence."""
        if not isinstance(other, Matrice):
            raise TypeError("Pas une matrice")
        if not self.nblignes == self.nbcolonnes == other.nblignes == other.nbcolonnes:
            raise RuntimeError("Pas la bonne taille")
        if threshold < 1:
            raise RuntimeError("Seuil trop bas")
        
        n = self.nblignes
        # Si on est en dessous du seuil de récursion, on applique la multiplication naïve
        if n <= threshold:
            return self * other
        # Si n est impair, on ajoute une ligne et une colonne de 0 et on applique récursivement
        # (ce cas n'était pas demandé dans le TD)
        elif n % 2 == 1:
            # Cette fonction crée une nouvelle matrice en ajoutant une ligne et une colonne de 0
            def augmenter(matrice):
                # La sytanxe liste[:] est une astuce classique python pour copier une liste
                return Matrice([ligne[:] + [0] for ligne in matrice.coeffs] + [[0]*(n+1)])
            # On appelle l'algorithme sur les matrices augmentées
            tmp = augmenter(self).strassen(augmenter(other), threshold)
            # On enlève la dernière ligne
            del tmp.coeffs[-1]
            # et la dernière colonne
            for ligne in tmp.coeffs:
                del ligne[-1]
            # On met à jour la taille de la matrice temporaire
            tmp.nblignes = tmp.nbcolonnes = n
            return tmp
        # Algorithme de Strassen
        else:
            m = n // 2
            # Cette fonction crée 4 sous-matrices de taille identique
            # Elle utilise lourdement la syntaxe liste[a:b], qui sélectionne 
            # la sous-liste contenant les éléments de a jusqu'à b-1.
            def blocs(matrice):
                a = Matrice([ligne[:m] for ligne in matrice.coeffs[:m]])
                b = Matrice([ligne[m:] for ligne in matrice.coeffs[:m]])
                c = Matrice([ligne[:m] for ligne in matrice.coeffs[m:]])
                d = Matrice([ligne[m:] for ligne in matrice.coeffs[m:]])
                return a, b, c, d
            a, b, c, d = blocs(self)
            x, y, z, t = blocs(other)
            # Appels récursifs
            q1 = a.strassen(x+z, threshold)
            q2 = d.strassen(y+t, threshold)
            q3 = (d-a).strassen(z-y, threshold)
            q4 = (b-d).strassen(z+t, threshold)
            q5 = (b-a).strassen(z, threshold)
            q6 = (c-a).strassen(x+y, threshold)
            q7 = (c-d).strassen(y, threshold)
            # Recombinaison du résultat
            A, B, C, D = q1+q5, q2+q3+q4-q5, q1+q3+q6-q7, q2+q7
            
            return Matrice([Aligne + Bligne for Aligne, Bligne in zip(A.coeffs, B.coeffs)]
                           + [Cligne + Dligne for Cligne, Dligne in zip(C.coeffs, D.coeffs)])

In [2]:
A = Matrice([[1,2,3],[4,5,6],[7,8,9]])
B = Matrice([[10,9,8],[7,6,5],[4,3,2]])
A == A, A == B, B == B

(True, False, True)

In [3]:
A+B

/ 11 11 11 \
| 11 11 11 |
\ 11 11 11 /

In [4]:
-B

/ -10  -9  -8 \
|  -7  -6  -5 |
\  -4  -3  -2 /

In [5]:
A-B

/ -9 -7 -5 \
| -3 -1  1 |
\  3  5  7 /

In [6]:
A*B

/  36  30  24 \
|  99  84  69 |
\ 162 138 114 /

In [7]:
A.strassen(B)

/  36  30  24 \
|  99  84  69 |
\ 162 138 114 /

## Tests

In [8]:
def rand_mat(lignes, colonnes, coeffs, min=1, max=1, sym=False):
    "Fonction d'utilité pour générer des matrices aléatoires"
    
    if (sym and lignes != colonnes):
        raise RuntimeError("Seules les matrices carrées peuvent être symétriques")
    if coeffs > lignes*colonnes:
        raise RuntimeError("Trop de coefficients")
    
    # On crée une liste avec autant d'éléments que la matrice à créer
    # Elle commence par des éléments entre min et max, et se termine par des 0
    from random import randint, sample
    nonzero = coeffs if not sym else coeffs // 2
    total = lignes*colonnes if not sym else lignes*(lignes+1) // 2
    entrees = [randint(min, max) for _ in range(nonzero)] + [0]*(total - nonzero)
    
    # On permute la liste avec l'algorithme de Fisher-Yates 
    # https://en.wikipedia.org/wiki/Fisher–Yates_shuffle
    for i in reversed(range(len(entrees))):
        j = randint(0,i)
        entrees[i], entrees[j] = entrees[j], entrees[i]
        
    if not sym:
        # On découpe en lignes
        mat = [entrees[i*colonnes : (i+1)*colonnes] for i in range(lignes)]
    else:
        # On remplit la partie triangulaire inférieure
        mat = [entrees[i*(i+1)//2 : (i+1)*(i+2)//2] + [None]*(lignes-i-1)
               for i in range(lignes)]
        # On symétrise
        for i in range(lignes):
            for j in range(i+1,lignes):
                mat[i][j] = mat[j][i]
        # On a été assez laxistes par rapport au paramètre coeffs
    return Matrice(mat)

Ici on fait des tests aléatoires pour vérifier que la méthode Strassen calcule le même résultat que la méthode naïve (cela prend quelques secondes).

In [9]:
def rand_dense_pair(n):
    return rand_mat(n, n, n**2, min=-10, max=10), rand_mat(n, n, n**2, min=-10, max=10)

for n in range(1,30):
    A, B = rand_dense_pair(n)
    assert A*B == A.strassen(B)

Ici on compare les temps d'éxécution pour $n$ qui grandit exponentiellement. On utilse la variante `%timeit -o`, qui permet de stocker les temps d'éxécution dans une variable

In [10]:
Tnaif, Tstrassen = [], []
for i in range(2,7):
    A, B = rand_dense_pair(2**i)
    print('2^%d:' % i)
    n = %timeit -o A*B
    s = %timeit -o A.strassen(B)
    Tnaif.append(n)
    Tstrassen.append(s)

2^2:
32.4 µs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.14 ms ± 2.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2^3:
166 µs ± 574 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
8.14 ms ± 51.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2^4:
1.05 ms ± 2.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
56.9 ms ± 774 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2^5:
7.6 ms ± 87.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
407 ms ± 4.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2^6:
58.4 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.96 s ± 69.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


On vérifie que la théorie de la complexité ne nous dit pas de bêtises. En effet, les temps d'exécutions de la méthode de Strassen sont multipliés par 7 à chaque doublement.

In [11]:
[ip1.best/i.best for i, ip1 in zip(Tstrassen, Tstrassen[1:])]

[7.096106310562845, 6.880017074058922, 7.245495599501129, 7.192190431150565]

La méthode naïve montre un comportement plus erratique, mais on voit qu'elle tend à une multiplcation par 8.

In [12]:
[ip1.best/i.best for i, ip1 in zip(Tnaif, Tnaif[1:])]

[5.186303104010197, 6.343409407126738, 7.18363896913009, 7.606923151839664]

In [13]:
[s.best / n.best for s, n in zip(Tstrassen, Tnaif)]

[35.87565905119527,
 49.08650448754937,
 53.238876336878086,
 53.69730381480276,
 50.76970372469321]

On voit que la méthode de Strassen est 50 fois plus lente que la méthode naïve pour $n=2^6$. À ce rythme, on s'attend à voir un croisement après $\log(50)/\log(8/7)$ doublements, c'est à dire...

In [14]:
import math
6 + math.log(50,8/7)

35.296653981797206

$n > 2^{35}$!! On rappelle qu'un ordinateur moyen a environ $2^{32}$ octets de mémoire. Autant dire que nous n'avons aucune chance d'observer ce croisement.

Heureusement, nous avons programmé l'algorithme de Strassen avec la possibilité d'arrêter la récursion en dessous d'un seuil.

In [15]:
Tnaif, Tstrassen = [], []
for i in range(6,10):
    A, B = rand_dense_pair(2**i)
    print('2^%d:' % i)
    n = %timeit -n 1 -r 1 -o A*B
    s = %timeit -n 1 -r 1 -o A.strassen(B,32)
    Tnaif.append(n)
    Tstrassen.append(s)

2^6:
59.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
55.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2^7:
450 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
398 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2^8:
3.54 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.83 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2^9:
31.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
20.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Alléluja! Nous avons un algorithme de Strassen qui bat l'algorithme naïf à partir de $n>64$!

In [16]:
[ip1.best/i.best for i, ip1 in zip(Tnaif, Tnaif[1:])]

[7.594093334520414, 7.876589782949883, 8.939963935651793]

In [17]:
[ip1.best/i.best for i, ip1 in zip(Tstrassen, Tstrassen[1:])]

[7.125741006395065, 7.098520785221912, 7.120611819214865]